# **Linear Models in TensorFlow: Timing Factors and Smart-Beta Strategies** 

**Project Goal**: To build a neural network, under linear models, that aims to predict the future returns of the momentum factor.

**Data Source**